# Object Detection with YOLO

This notebook provides a ready-to-run setup of the You Only Look Once (YOLO) v3 network for object detection. The [YOLO family of models](https://www.cv-foundation.org/openaccess/content_cvpr_2016/html/Redmon_You_Only_Look_CVPR_2016_paper.html) were created by [Joseph Chet Redmon](https://pjreddie.com/).  Training these models requires large data sets like [ImageNet]() and [Microsoft COCO]() and significant compute resources, making it infeasible for most users to train their own models.  Thankfully, the researchers have released the weights of their trained models and the community has developed code that allows these models to be run with frameworks like Pytorch and Tensorflow.  This notebook uses [code](https://github.com/experiencor/keras-yolo3) released under the MIT license by [Huynh Ngoc Anh](https://github.com/experiencor) to run the pre-trained YOLO model in Keras.

To use this notebook, you will need to download the YOLOv3 model weights from [https://pjreddie.com/media/files/yolov3.weights](https://pjreddie.com/media/files/yolov3.weights).  Place the `yolov3.weights` file in the same directory as this notebook.

You will need to change the `input_image_path` and `output_image_path` parameters.

In [81]:
import cv2
import yolov3

In [82]:
# Parameters

weights_path = "yolov3.weights"

# Supports .png, .jpg, and .jpeg files
filename = 'food'
input_image_path = f"input/{filename}.jpg"
output_image_path = f"output/{filename}_detected.jpg"


net_h, net_w = 416, 416
obj_thresh, nms_thresh = 0.5, 0.45
anchors = [[116,90,  156,198,  373,326],  [30,61, 62,45,  59,119], [10,13,  16,30,  33,23]]
labels = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", \
          "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", \
          "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", \
          "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", \
          "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", \
          "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", \
          "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", \
          "chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse", \
          "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", \
          "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]



In [83]:
# preprocess the image
image = cv2.imread(input_image_path)
image_h, image_w, _ = image.shape
new_image = yolov3.preprocess_input(image, net_h, net_w)

In [84]:
# make the yolov3 model to predict 80 classes on COCO
model = yolov3.make_yolov3_model()

# load the weights trained on COCO into the model
weight_reader = yolov3.WeightReader(weights_path)
weight_reader.load_weights(model)

loading weights of convolution #0
loading weights of convolution #1
loading weights of convolution #2
loading weights of convolution #3
no convolution #4
loading weights of convolution #5
loading weights of convolution #6
loading weights of convolution #7
no convolution #8
loading weights of convolution #9
loading weights of convolution #10
no convolution #11
loading weights of convolution #12
loading weights of convolution #13
loading weights of convolution #14
no convolution #15
loading weights of convolution #16
loading weights of convolution #17
no convolution #18
loading weights of convolution #19
loading weights of convolution #20
no convolution #21
loading weights of convolution #22
loading weights of convolution #23
no convolution #24
loading weights of convolution #25
loading weights of convolution #26
no convolution #27
loading weights of convolution #28
loading weights of convolution #29
no convolution #30
loading weights of convolution #31
loading weights of convolution #32

In [85]:
# run the prediction
yolos = model.predict(new_image)

In [86]:
boxes = []

for i in range(len(yolos)):
    # decode the output of the network
    boxes += yolov3.decode_netout(yolos[i][0], anchors[i], obj_thresh, nms_thresh, net_h, net_w)

# correct the sizes of the bounding boxes
yolov3.correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w)

# suppress non-maximal boxes
yolov3.do_nms(boxes, nms_thresh)     

# draw bounding boxes on the image using labels
yolov3.draw_boxes(image, boxes, labels, obj_thresh) 

apple: 65.51755666732788%
sandwich: 82.17741250991821%
donut: 84.79660749435425%
donut: 84.63027477264404%


array([[[ 13,   9,   4],
        [ 13,   9,   4],
        [ 13,   9,   4],
        ...,
        [121,  77,  24],
        [121,  77,  24],
        [121,  77,  24]],

       [[ 13,   9,   4],
        [ 13,   9,   4],
        [ 13,   9,   4],
        ...,
        [121,  77,  24],
        [121,  77,  24],
        [121,  77,  24]],

       [[ 13,   9,   4],
        [ 13,   9,   4],
        [ 13,   9,   4],
        ...,
        [121,  77,  24],
        [121,  77,  24],
        [121,  77,  24]],

       ...,

       [[  0,  24,  30],
        [  1,  25,  31],
        [  0,  26,  32],
        ...,
        [ 54,  65,  79],
        [ 52,  65,  79],
        [ 52,  65,  79]],

       [[  2,  26,  32],
        [  2,  26,  32],
        [  0,  26,  32],
        ...,
        [ 53,  64,  78],
        [ 51,  64,  78],
        [ 51,  64,  78]],

       [[  2,  26,  32],
        [  3,  27,  33],
        [  1,  27,  33],
        ...,
        [ 53,  64,  78],
        [ 51,  64,  78],
        [ 51,  64,  78]]

In [87]:
# write out the resulting image with object annotations
cv2.imwrite(output_image_path, (image).astype('uint8')) 

True